In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import re
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from scipy.sparse import coo_matrix, hstack
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
xgb_classifier = xgb.XGBClassifier()


In [2]:
embeddings_index = {}
with open('glove/glove.twitter.27B.200d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [3]:
def get_mean_embeddings(texts,embeddings):
    means = []
    dim = len(list(embeddings.values())[0])
    for text in texts :
        text = nltk.WordPunctTokenizer().tokenize(text)
        means.append(np.mean([embeddings[w] if w in embeddings else np.zeros(dim) for w in text], axis=0))
    return np.array(means)

In [33]:
DATAFRAME_vect = pd.read_csv('AMI2020_TrainingSet/AMI2020_training_raw.tsv', sep='\t')
vectorizer = TfidfVectorizer(min_df=3,  max_features=None, 
    strip_accents='unicode', analyzer='word', token_pattern=r'\b[^\d\W]+\b',
    ngram_range=(2, 3),use_idf=True)
DATAFRAME_test=

In [34]:
DATAFRAME_emb=pd.read_csv('different processed dataframes/noun_chuncks,processed.tsv',sep='\t')

In [35]:
import random
def run_cv(k_fold, data,label,text_data=None):
    accuracy_scores = f1_scores = confusion_matrices = []
    labels = np.array(label)
    skf = StratifiedKFold(k_fold)
    cv_splits = skf.split(data,labels)
    min_inidices = ([],[])
    min_acc = 100
    media=0
    for train, test in cv_splits:
        traindata,y_train,= data[train],labels[train]
        testdata,y_test=data[test],labels[test]
        
        model = LogisticRegression(penalty='l2', dual=True, tol=0.0001, max_iter=100000,
                         C=3, fit_intercept=True, intercept_scaling=1.0, 
                         solver = 'liblinear', warm_start=False,
                         class_weight=None, random_state=None)
      
        '''
        traintext,testtext=text_data[train],text_data[test]
        train_emb,test_emb=get_mean_embeddings(traintext,embeddings_index),get_mean_embeddings(testtext,embeddings_index)
        traindata=hstack((train_emb,traindata))
        testdata=hstack((test_emb,testdata))
        '''
        model.fit(traindata,y_train)
        result=model.predict(testdata)
        score = accuracy_score(y_test, result)
        accuracy_scores.append(score)
        if score < min_acc:
            min_acc = score
            split_inidices = (train, test)
        f1sc = f1_score(y_test, result, average='weighted')
        media=media+f1sc
        print('f1score:',f1sc)
        f1_scores.append(f1sc)
    print (f'min cv acc:{min_acc}\nmedia:{media}')
    print(np.mean(f1_scores))
